# Dataset Statistics


In [ ]:
import pandas as pd

data = pd.read_csv("data/SHS-YT.csv", sep=";")


In [ ]:
import utils

utils.get_mean_sim("coverhunter")

In [ ]:
data.groupby(["set_id", "nlabel"]).describe().reset_index().groupby("nlabel").describe()


### Agreement

In [ ]:
import krippendorff
import pandas as pd


def krippendorff_pivot(df_hit_pivot: pd.DataFrame):
    import krippendorff
    df_hit_pivot = df_hit_pivot[[f"worker_ind{i}" for i in range(5)]]
    try:
        return round(krippendorff.alpha(df_hit_pivot.T, level_of_measurement='ordinal'), 2)
    except AssertionError:
        return None
    except ValueError:
        return None
    
krippendorff_pivot(data[[f"worker_ind{i}" for i in range(5)]])


In [ ]:
def rename_sample_groups(x):
    if x != x: # this works to detect nans
        return x
    else:
        return x.replace("re-move_favs", 
                         "DisAgrMus").replace("ditto_favs", 
                                     "DisAgrTxt").replace("mutual_unconfident", "MutUnc")

# Create a list of worker columns
worker_cols = [f'worker_ind{i}' for i in range(5)]

# Create a new dataframe with the relevant columns
pivot_data = data[worker_cols + ['sample_group', 'label']]

# Group by 'sample_group' and 'mv_nlabel' and count occurrences
pivot_table = pivot_data.groupby(['sample_group', 'label']).size().unstack(fill_value=0)

custom_sort_order = ['Match', 'Version', 'Other', 'No Music']

# Reindex the DataFrame based on the custom sort order
pivot_table = pivot_table.reindex(columns=custom_sort_order)
  
for sample_group in data.sample_group.unique():
    pivot_table.loc[sample_group, "Agreement"] = krippendorff_pivot(data.query(f"sample_group == '{sample_group}'"))
    pivot_table.loc[sample_group, "Curated"] = int(data.query(f"sample_group == '{sample_group}'").label_expert.count())
    pivot_table.loc[sample_group, "Relabeled"] = int(len(data.query(f"sample_group == '{sample_group}' & ((~nlabel_expert.isna() & nlabel_expert != nlabel_mturk))")))

for label in data.label.unique():
    pivot_table.loc["Curated", label] = int(len(data.query(f"label == '{label}' & (origin == 'expert')")))
    pivot_table.loc["Relabeled", label] = int(len(data.query(f"label == '{label}' & ((~nlabel_expert.isna() & nlabel_expert != nlabel_mturk))")))

pivot_table.index = pivot_table.index.to_series().apply(rename_sample_groups).fillna("None")
pivot_table = pivot_table.rename_axis("Group").rename_axis("Label", axis=1)
pivot_table




### Overlaps with existing Datasets

#### Da-Tacos

In [ ]:
data_datacos = pd.read_csv("data/da-tacos.csv", sep=";")
data[data.yt_id.isin(data_datacos.yt_id)]


#### SHS100K

In [ ]:
data_shs100k = pd.read_csv("data/shs100k.csv", sep=";")
pd.merge(data[["set_id", "yt_id", "label"]], 
         data_shs100k[["set_id", "yt_id", "title", "subset"]], 
         on="yt_id", suffixes=["_YT", "_SHS"])

